In [ ]:
#####################################################
# PLEASE RUN THIS FIRST OTHERWISE NOTHING WILL WORK #
#####################################################


import os
import shutil

directory = "/content/Math-4997-Summer-23"
shutil.rmtree(directory)
!git clone https://github.com/skylarwilson/Math-4997-Summer-23
print("\nFILE UPLOADING DONE, YOU MAY PROCEED.")

In [ ]:
###########################################
# THIS IS THE MODEL OF THE NEURAL NETWORK #
# PLEASE RUN THIS NEXT                    #
#                                         #
# IF YOU MADE CHANGES TO THE MODEL        #
# PLEASE RERUN THIS CODE                  #
###########################################

# Model Definition
import torch
import torch.nn as nn
import torch.nn.functional as F

# Model Definition
class CountingModel(nn.Module):
    def __init__(self):
        super(CountingModel, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 484, 8)  # Adjusted input size
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        # Apply convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        # Determine the output size after pooling
        _, _, height, width = x.size()
        output_size = height * width * 128

        # Flatten the tensor
        x = x.view(-1, output_size)

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
print("MODEL LOADED, YOU MAY PROCEED")

In [ ]:
#####################################################
# THIS IS THE TRAINING CODE; YOU CAN SAFELY CHANGE: #
#                                                   #
# batch_size                                        #
# learning_rate                                     #
# num_epochs                                        #
#                                                   #
# IF YOU MADE CHANGES TO THIS CODE, RERUN IT BEFORE #
# USING THE TESTING CODE                            #
#####################################################


import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import json
import os
import torch.cuda as cuda
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

device = torch.device('cuda')

# Define dataset
class CustomDataset(data.Dataset):
    def __init__(self, image_folder, json_file, transform=None):
        with open(json_file) as f:
            self.data_info = json.load(f)
        self.image_keys = list(self.data_info.keys())
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        # Access the data using the keys
        image_key = self.image_keys[idx]
        img_name = os.path.join(self.image_folder, self.data_info[image_key]["file_name"])
        image = Image.open(img_name).convert('RGB')
        num_dots = self.data_info[image_key]["num_dots"]

        if self.transform:
            image = self.transform(image)

        return image, num_dots



# Configuration
image_folder = "/content/Math-4997-Summer-23/eggCounter/Fake eggs"
json_file = "/content/Math-4997-Summer-23/eggCounter/labels.json"



####################################
# Feel free to change these values #
batch_size = 32
learning_rate = 0.008
num_epochs = 10
####################################



# Transformations
transform = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Load Dataset
dataset = CustomDataset(image_folder=image_folder, json_file=json_file, transform=transform)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss and optimizer
model = CountingModel()
model.to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, num_dots in dataloader:
        images, num_dots = images.to(device), num_dots.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, num_dots.float().unsqueeze(1))

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

# Save the model
model_state_dict = model.state_dict()

# Define the model name
model_name = 'counting_model.ckpt'

# Provide the file path to save the model state dictionary
file_path = f'/content/Math-4997-Summer-23/eggCounter/{model_name}'
torch.save(model_state_dict, file_path)

print("\nMODEL SAVED, YOU CAN PROCEED TO TEST")

In [ ]:
########################################################
# THIS IS TO TEST THE MODEL; IT HAS 4 IMAGES AS INPUTS #
#                                                      #
# 27-real AND 84-real ARE OUR REAL PICTURES OF EGGS    #
#                                                      #
# 120-dots AND 27-dots ARE SIMULATED IMAGES OF EGGS    #
# THAT WERE USED IN THE TRAINING MODEL                 #
#                                                      #
# THERE IS NOTHING TO BE CHANGED HERE                  #
########################################################



import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

# Load the trained model
model = CountingModel()
model.load_state_dict(torch.load("/content/Math-4997-Summer-23/eggCounter/counting_model.ckpt"))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((360, 360)),
    transforms.ToTensor()
])

# List of available image paths
image_paths = [
    "/content/Math-4997-Summer-23/eggCounter/27-real.jpg",
    "/content/Math-4997-Summer-23/eggCounter/84-real.jpg",
    "/content/Math-4997-Summer-23/eggCounter/120-dots.png",
    "/content/Math-4997-Summer-23/eggCounter/27-dots.png"
]

# Iterate over the image paths
for image_path in image_paths:
    # Extract the image name from the file path
    image_name = os.path.basename(image_path)

    # Load and transform the test image
    test_image = Image.open(image_path).convert('RGB')
    test_image = transform(test_image).unsqueeze(0)  # Add batch dimension

    # Predict the number of dots
    with torch.no_grad():
        output = model(test_image)

    predicted_count = int(torch.round(output.squeeze()))

    print("Image:\t", image_name)
    print("Predicted Dot Count:", predicted_count)
    print("-------------------\n")

print("\nIF YOU GET THE SAME NUMBER FOR PREDICTED DOT COUNT")
print("PLEASE RERUN THE TRAINING CODE TO MAKE A NEW MODEL")